In [1]:
#setting up the environment

import pandas as pd
import matplotlib.pyplot as plt
import trxtools as tt
import os
from pyCRAC.Parsers import GTF2
import pyBigWig

pathTAB = "../seq_references/Saccharomyces_cerevisiae.EF4.74.dna.toplevel.shortChrNames.tab"
pathGTF = "../seq_references/Saccharomyces_cerevisiae.EF4.74.shortChNames_with_PolIII_transcripts_extended_slop_intergenic_sort.gtf"
gtf = GTF2.Parse_GTF()
gtf.read_GTF(pathGTF)
gtf.read_TAB(pathTAB)

sequences = pd.read_csv(pathTAB, sep='\t',names=['chr','sequence'],index_col=0)
chr_len = sequences['sequence'].str.len()

pathDir = "../04_BigWig/"

In [2]:
data_files_raw = [f for f in os.listdir(pathDir) if 'raw' in f and "bw" in f]
data_files_5end = [f for f in os.listdir(pathDir) if 'PROFILE_5end' in f and "bw" in f]
data_files_3end = [f for f in os.listdir(pathDir) if 'PROFILE_3end' in f and "polyA" not in f and "bw" in f]
data_files_polyA = [f for f in os.listdir(pathDir) if 'PROFILE_3end' in f and "polyA" in f and "bw" in f]

# number of input datasets
# print(len(os.listdir(pathDir)))
# print(len(data_files_raw))
# print(len(data_files_5end))
# print(len(data_files_3end))
# print(len(data_files_polyA))

# print positions of rDNA in yeast genome

# print(gtf.strand("RDN37-1"), gtf.chromosome("RDN37-1"), gtf.chromosomeCoordinates("RDN37-1"))
# print(gtf.strand("RDN37-2"), gtf.chromosome("RDN37-2"), gtf.chromosomeCoordinates("RDN37-2"))

597
120
120
120
117


In [16]:
def strip_BigWig_names(files=list()):
    #returns uniq names for *fwd.bw and *rev.bw files
    return list(set([f.replace("_fwd.bw","").replace("_rev.bw","") for f in files]))

def getSeqData(gene_name, data_path, name, gtf, ranges=0):
    strand, chromosome, coordinates = gtf.strand(gene_name), gtf.chromosome(gene_name), gtf.chromosomeCoordinates(gene_name)
    if strand == "+":
        bw = pyBigWig.open(data_path+name+"_fwd.bw")
        return pd.Series(bw.values(chromosome,min(coordinates)-ranges,max(coordinates)+ranges))
    if strand == "-":
        bw = pyBigWig.open(data_path+name+"_rev.bw")
        return pd.Series(bw.values(chromosome,min(coordinates)-ranges,max(coordinates)+ranges)[::-1])

def geneFromBigWig(gene_name, data_path, data_files, gtf, ranges=0):
    df_t1 = pd.DataFrame()
    df_t1["nucleotide"] = "_".join(gtf.genomicSequence(gene_name,ranges=ranges)).split("_")
    for name in strip_BigWig_names(data_files):
        df_t1[name] = getSeqData(gene_name, data_path, name, gtf, ranges=ranges)
    return df_t1

df01_RDN37_datasets_3end = geneFromBigWig(gene_name="RDN37-1", data_path=pathDir, data_files=data_files_3end, gtf=gtf,ranges=300)
# df01_RDN37_datasets_3end.fillna(0.0).plot()

In [59]:
def getTotalCountBigWig(chrom={},data_path="", data_files=""):
    df_output = pd.DataFrame(index=chrom.keys())
    for name in strip_BigWig_names(data_files):
        df_temp = pd.DataFrame(index=chrom.keys())
        try:
            bw = pyBigWig.open(data_path+name+"_fwd.bw")
            for c in chrom.keys():
                df_temp.loc[c,'fwd']= bw.stats(c,type='sum',exact=True)[0]
        except:
            df_temp.loc[c,'fwd']= 0.0
        try:
            bw = pyBigWig.open(data_path+name+"_rev.bw")
            for c in chrom.keys():
                df_temp.loc[c,'rev']= bw.stats(c,type='sum',exact=True)[0]
        except:
            df_temp.loc[c,'rev']= 0.0
        
        df_output[name] = df_temp.sum(1)
    return df_output

df01_3ends = getTotalCountBigWig(chrom=chr_len.sort_index().to_dict(),data_path=pathDir,data_files=data_files_3end)
df02_polyA = getTotalCountBigWig(chrom=chr_len.sort_index().to_dict(),data_path=pathDir,data_files=data_files_polyA)

[urlOpen] Couldn't open ../04_BigWig/XX000000_Rpa190HTP_wt_none_6_PROFILE_3end_polyA_rev.bw for reading
[urlOpen] Couldn't open ../04_BigWig/XX000000_Rpa190HTP_wt_none_6_PROFILE_3end_polyA_rev.bw for reading
[pyBwOpen] bw is NULL!
[urlOpen] Couldn't open ../04_BigWig/EP190925_Rpa135HTP_wt_noUV_none_1_PROFILE_3end_polyA_rev.bw for reading
[urlOpen] Couldn't open ../04_BigWig/EP190925_Rpa135HTP_wt_noUV_none_1_PROFILE_3end_polyA_rev.bw for reading
[pyBwOpen] bw is NULL!
[urlOpen] Couldn't open ../04_BigWig/EP160126_Rpa190HTP_hmo1d_none_2_PROFILE_3end_polyA_rev.bw for reading
[urlOpen] Couldn't open ../04_BigWig/EP160126_Rpa190HTP_hmo1d_none_2_PROFILE_3end_polyA_rev.bw for reading
[pyBwOpen] bw is NULL!


In [68]:
df02_polyA.sum().sort_values()

XX000000_Rpa190HTP_wt_80s_1_PROFILE_3end_polyA                        0.0
XX000000_Rpa190HTP_wt_100s_1_PROFILE_3end_polyA                       0.0
C20P3-TT171005_Rpa135HTP_wt_none_x_PROFILE_3end_polyA                 0.0
EP190925_Rpa135HTP_wt_noUV_none_1_PROFILE_3end_polyA                  0.0
XX000000_Rpa190HTP_wt_80s_2_PROFILE_3end_polyA                        0.0
XX000000_Rpa190HTP_wt_none_6_PROFILE_3end_polyA                       0.0
C22P2beads-TT180115_FPHRpa12_wt_none_1_PROFILE_3end_polyA             0.0
C18P2-TT170123_Rpa135HTP_wt_none_1_PROFILE_3end_polyA                 0.0
XX000000_Rpa190HTP_wt_none_3_PROFILE_3end_polyA                       0.0
C23P2-TT181022_Rpa135HTP_wt_none_2_PROFILE_3end_polyA                 0.0
XX000000_Rpa190HTP_wt_none_5_PROFILE_3end_polyA                       0.0
EP190925_Rpa135HTP_Rpa12dC_none_2_PROFILE_3end_polyA                  0.0
CDF000002_Rrp41HTP_Rrp44-exo_none_2_PROFILE_3end_polyA                0.0
EP160126_Rpa190HTP_hmo1d_none_2_PROFIL